# Lab 2: Applied Optimization (TensorFlow)

In this lab, we will apply Regularization (L2, Dropout), Optimization (Adam), and Batch Normalization to a CNN for CIFAR-10.


### Step 0: Setup and Data Loading


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

# Load CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize
train_images, test_images = train_images / 255.0, test_images / 255.0


### Step 1: Baseline Model (Likely High Variance)


In [ ]:
def build_baseline_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))
    return model

baseline_model = build_baseline_model()
baseline_model.compile(optimizer='sgd',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

history_base = baseline_model.fit(train_images, train_labels, epochs=10, 
                                  validation_data=(test_images, test_labels), verbose=1)


### Step 2: Adding Regularization (L2 + Dropout)


In [ ]:
def build_reg_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))
    return model

reg_model = build_reg_model()
reg_model.compile(optimizer='sgd',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

history_reg = reg_model.fit(train_images, train_labels, epochs=10, 
                            validation_data=(test_images, test_labels), verbose=1)


### Step 3: Adding Adam Optimizer


In [ ]:
adam_model = build_reg_model()
adam_model.compile(optimizer='adam',
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

history_adam = adam_model.fit(train_images, train_labels, epochs=10, 
                              validation_data=(test_images, test_labels), verbose=1)


### Step 4: Adding Batch Normalization


In [ ]:
def build_bn_model():
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dense(10))
    return model

bn_model = build_bn_model()
bn_model.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

history_bn = bn_model.fit(train_images, train_labels, epochs=10, 
                          validation_data=(test_images, test_labels), verbose=1)


### Step 5: Compare Results


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history_base.history['val_accuracy'], label='Baseline')
plt.plot(history_reg.history['val_accuracy'], label='L2 + Dropout')
plt.plot(history_adam.history['val_accuracy'], label='L2 + Dropout + Adam')
plt.plot(history_bn.history['val_accuracy'], label='Full Model (BN + Adam)')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
